In [1]:
import numpy as np
import os
import shutil
import subprocess
import pandas as pd


In [2]:

%%file node_info.csv
node_id,region,ServerOrClientregion,cluster_id,key_file,clientID,ServerID, PublicIp
0, 'us-west1-b', 0, 0, 'FKA.pem', -1, 0,
1, 'us-west1-b', 0, 0, 'FKA.pem', -1, 1,
2, 'us-west1-b', 0, 0, 'FKA.pem', -1, 2,
3, 'us-west1-b', 0, 0, 'FKA.pem', -1, 3,
4, 'us-west1-b', 0, 1, 'FKA.pem', -1, 4,
5, 'us-west1-b', 0, 1, 'FKA.pem', -1, 5,
6, 'us-west1-b', 0, 1, 'FKA.pem', -1, 6,
7, 'us-west1-b', 0, 1, 'FKA.pem', -1, 7,
8, 'us-west1-b', 0, 2, 'FKA.pem', -1, 8,
9, 'us-west1-b', 0, 2, 'FKA.pem', -1, 9,
10, 'us-west1-b', 0, 2, 'FKA.pem', -1, 10,
11, 'us-west1-b', 0, 2, 'FKA.pem', -1, 11,
12, 'us-west1-b', 0, 3, 'FKA.pem', -1, 12,
13, 'us-west1-b', 0, 3, 'FKA.pem', -1, 13,
14, 'us-west1-b', 0, 3, 'FKA.pem', -1, 14,
15, 'us-west1-b', 0, 3, 'FKA.pem', -1, 15,
16, 'us-west1-b', 1, -1, 'FKA.pem', 0, -1,
17, 'us-west1-b', 1, -1, 'FKA.pem', 1, -1,
18, 'us-west1-b', 1, -1, 'FKA.pem', 2, -1,
19, 'us-west1-b', 1, -1, 'FKA.pem', 3, -1,
20, 'us-west1-b', 0, 4, 'FKE.pem', -1, 16,
21, 'us-west1-b', 0, 4, 'FKE.pem', -1, 17,
22, 'us-west1-b', 0, 4, 'FKE.pem', -1, 18,
23, 'us-west1-b', 0, 4, 'FKE.pem', -1, 19,
24, 'us-west1-b', 0, 5, 'FKE.pem', -1, 20,
25, 'us-west1-b', 0, 5, 'FKE.pem', -1, 21,
26, 'us-west1-b', 0, 5, 'FKE.pem', -1, 22,
27, 'us-west1-b', 0, 5, 'FKE.pem', -1, 23,
28, 'us-west1-b', 0, 6, 'FKE.pem', -1, 24,
29, 'us-west1-b', 0, 6, 'FKE.pem', -1, 25,
30, 'us-west1-b', 0, 6, 'FKE.pem', -1, 26,
31, 'us-west1-b', 0, 6, 'FKE.pem', -1, 27,
32, 'us-west1-b', 0, 7, 'FKE.pem', -1, 28,
33, 'us-west1-b', 0, 7, 'FKE.pem', -1, 29,
34, 'us-west1-b', 0, 7, 'FKE.pem', -1, 30,
35, 'us-west1-b', 0, 7, 'FKE.pem', -1, 31,
36, 'us-west1-b', 1, -1, 'FKE.pem', 4, -1,
37, 'us-west1-b', 1, -1, 'FKE.pem', 5, -1,
38, 'us-west1-b', 1, -1, 'FKE.pem', 6, -1,
39, 'us-west1-b', 1, -1, 'FKE.pem', 7, -1,
40, 'us-west1-b', 0, 8, 'FKU2.pem', -1, 32,
41, 'us-west1-b', 0, 8, 'FKU2.pem', -1, 33,
42, 'us-west1-b', 0, 8, 'FKU2.pem', -1, 34,
43, 'us-west1-b', 0, 8, 'FKU2.pem', -1, 35,
44, 'us-west1-b', 0, 9, 'FKU2.pem', -1, 36,
45, 'us-west1-b', 0, 9, 'FKU2.pem', -1, 37,
46, 'us-west1-b', 0, 9, 'FKU2.pem', -1, 38,
47, 'us-west1-b', 0, 9, 'FKU2.pem', -1, 39,
48, 'us-west1-b', 0, 10, 'FKU2.pem', -1, 40,
49, 'us-west1-b', 0, 10, 'FKU2.pem', -1, 41,
50, 'us-west1-b', 0, 10, 'FKU2.pem', -1, 42,
51, 'us-west1-b', 0, 10, 'FKU2.pem', -1, 43,
52, 'us-west1-b', 0, 11, 'FKU2.pem', -1, 44,
53, 'us-west1-b', 0, 11, 'FKU2.pem', -1, 45,
54, 'us-west1-b', 0, 11, 'FKU2.pem', -1, 46,
55, 'us-west1-b', 0, 11, 'FKU2.pem', -1, 47,
56, 'us-west1-b', 1, -1, 'FKU2.pem', 8, -1,
57, 'us-west1-b', 1, -1, 'FKU2.pem', 9, -1,
58, 'us-west1-b', 1, -1, 'FKU2.pem', 10, -1,
59, 'us-west1-b', 1, -1, 'FKU2.pem', 11, -1,


Overwriting node_info.csv


In [3]:
df =pd.read_csv('node_info.csv')

node_info_dict = df.to_dict()

node_info_dict

{'node_id': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49,
  50: 50,
  51: 51,
  52: 52,
  53: 53,
  54: 54,
  55: 55,
  56: 56,
  57: 57,
  58: 58,
  59: 59},
 'region': {0: " 'us-west1-b'",
  1: " 'us-west1-b'",
  2: " 'us-west1-b'",
  3: " 'us-west1-b'",
  4: " 'us-west1-b'",
  5: " 'us-west1-b'",
  6: " 'us-west1-b'",
  7: " 'us-west1-b'",
  8: " 'us-west1-b'",
  9: " 'us-west1-b'",
  10: " 'us-west1-b'",
  11: " 'us-west1-b'",
  12: " 'us-west1-b'",
  13: " 'us-west1-b'",
  14: " 'us-west1-b'",
  15: " 'us-west1-b'",
  16: " 'us-west1-b'",
  17: " 'us-west1

In [4]:
total_count = 0

for key in node_info_dict['ServerOrClientregion'].keys():
    if node_info_dict['ServerOrClientregion'][key] ==0:
        total_count = total_count + 1
        
total_count

48

In [5]:
replica_count = 0

for key in node_info_dict['cluster_id'].keys():
    if node_info_dict['cluster_id'][key] ==0:
        replica_count = replica_count + 1
        
        
replica_count       

4

In [6]:
n_clusters = int(total_count / replica_count)

nclients = n_clusters

In [7]:
clusters_tracked = []
cluster_regions_tracked = []

for key in node_info_dict['region']:
    if int(node_info_dict['cluster_id'][key]) not in clusters_tracked and int(node_info_dict['cluster_id'][key]) >=0:
        clusters_tracked.append(int(node_info_dict['cluster_id'][key]))
        cluster_regions_tracked.append(node_info_dict['region'][key].strip().replace('"', '')[1:-1])
        
        
cluster_regions_tracked     

['us-west1-b',
 'us-west1-b',
 'us-west1-b',
 'us-west1-b',
 'us-west1-b',
 'us-west1-b',
 'us-west1-b',
 'us-west1-b',
 'us-west1-b',
 'us-west1-b',
 'us-west1-b',
 'us-west1-b']

In [8]:
cluster_regions_tracked, clusters_tracked

(['us-west1-b',
  'us-west1-b',
  'us-west1-b',
  'us-west1-b',
  'us-west1-b',
  'us-west1-b',
  'us-west1-b',
  'us-west1-b',
  'us-west1-b',
  'us-west1-b',
  'us-west1-b',
  'us-west1-b'],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

In [9]:

regions = cluster_regions_tracked


In [10]:
# regions = [ 'us-west1-b','us-west1-b', 'us-west1-b','us-west1-b',\
#            'europe-west3-c', 'europe-west3-c', 'europe-west3-c', 'europe-west3-c', \
#            'asia-south1-c', 'asia-south1-c', 'asia-south1-c', 'asia-south1-c']


# regions = [ 'us-west1-b','us-west1-b', 'us-west1-b','us-west1-b']


# regions = [ 'us-west1-b','us-west1-b', 'us-west1-b','us-west1-b',\
#            'us-west1-b', 'us-west1-b', 'us-west1-b', 'us-west1-b', \
#            'us-west1-b', 'us-west1-b', 'us-west1-b', 'us-west1-b']



# regions = [ 'europe-west3-c', 'asia-south1-c']

# regions = [ 'us-west1-b', 'us-west1-b']



# regions = [ 'us-west1-b', 'us-west1-b','us-west1-b','us-west1-b', 'us-west1-b', 'us-west1-b','us-west1-b','us-west1-b', 'us-west1-b', 'us-west1-b','us-west1-b','us-west1-b']
# regions = [ 'us-west1-b', 'us-west1-b', 'us-west1-b', 'us-west1-b']



# regions = ['europe-west3-c',  \
#            'asia-south1-c']


n_regions = len(regions)

In [11]:
for i,k in enumerate(regions):
    print(i,k)

0 us-west1-b
1 us-west1-b
2 us-west1-b
3 us-west1-b
4 us-west1-b
5 us-west1-b
6 us-west1-b
7 us-west1-b
8 us-west1-b
9 us-west1-b
10 us-west1-b
11 us-west1-b


In [12]:
lines  = []
zones = {}


lim = 0
for reg in regions:
    for i in range(replica_count+int(nclients/n_clusters)):
        line = 'gcloud compute instances create resdb-'+f"{lim+i:02}"+' --project=ucr-ursa-major-lesani-lab --zone='+reg+' --machine-type=e2-highmem-2 --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=goog-ec-src=vm_add-gcloud --reservation-affinity=any'
        
        print(line)
        lines.append(line)
        zones[i] = reg
    
    lim = lim + replica_count+int(nclients/n_clusters)
        
    
    
    
with open('launch_cloud_instances.sh', 'w') as f:
    for item in lines:
        f.write(str(item) + '\n')


gcloud compute instances create resdb-00 --project=ucr-ursa-major-lesani-lab --zone=us-west1-b --machine-type=e2-highmem-2 --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=g

In [13]:
import subprocess
import concurrent.futures

def run_command(command):
    subprocess.call(command, shell=True)

commands = lines

# Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
with concurrent.futures.ThreadPoolExecutor(max_workers=len(commands)) as executor:
    # Submit each command to the executor for parallel execution
    futures = [executor.submit(run_command, command) for command in commands]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)


ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-28' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-56' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-13' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-11' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-50' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/inst

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-40' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-22' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-31' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-33' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-00' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/inst

In [14]:
# os.system('sh launch_cloud_instances.sh')

In [15]:
for folder in os.listdir('.'):
    if 'config' in folder and folder!='config' and 'zip' not in folder and '.' not in folder:
        shutil.rmtree(folder)

In [16]:
# os.system("aws --region ap-south-1 ec2 describe-instances --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PrivateIpAddress]' --output text > all_internal_ips")

# os.system("aws --region ap-south-1 ec2 describe-instances  --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PublicIpAddress]' --output text > all_external_ips")


instance_type = 'e2-small'


redirect = '>'

os.system('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)" '+\
          redirect+' all_internal_ips')



os.system('gcloud compute instances list --format="value(networkInterfaces[0].networkIP)" '+\
      redirect+' all_external_ips')












0

In [17]:
with open('all_internal_ips') as f:
    ipList = f.read().splitlines()
ipList = [x for x in ipList if x!='None']

In [18]:
ipList, len(ipList)

(['10.138.0.85',
  '10.138.0.126',
  '10.138.0.96',
  '10.138.0.109',
  '10.138.0.101',
  '10.138.0.79',
  '10.138.0.74',
  '10.138.0.69',
  '10.138.0.122',
  '10.138.0.80',
  '10.138.0.64',
  '10.138.0.87',
  '10.138.15.225',
  '10.138.0.73',
  '10.138.0.98',
  '10.138.15.201',
  '10.138.15.194',
  '10.138.0.110',
  '10.138.0.103',
  '10.138.15.219',
  '10.138.0.107',
  '10.138.15.202',
  '10.138.0.114',
  '10.138.15.213',
  '10.138.0.84',
  '10.138.0.75',
  '10.138.0.81',
  '10.138.0.102',
  '10.138.0.77',
  '10.138.0.71',
  '10.138.15.221',
  '10.138.0.105',
  '10.138.0.104',
  '10.138.0.100',
  '10.138.0.70',
  '10.138.0.95',
  '10.138.0.99',
  '10.138.15.217',
  '10.138.0.68',
  '10.138.0.106',
  '10.138.15.214',
  '10.138.15.215',
  '10.138.0.97',
  '10.138.0.111',
  '10.138.0.65',
  '10.138.15.195',
  '10.138.0.108',
  '10.138.0.78',
  '10.138.0.72',
  '10.138.0.82',
  '10.138.0.121',
  '10.138.0.124',
  '10.138.0.67',
  '10.138.0.83',
  '10.138.0.66',
  '10.138.15.216',
  '10.1

In [19]:
new_server_only_ips= []
new_client_only_ips= []



for i in range(total_count + nclients):
    if node_info_dict['ServerOrClientregion'][i] ==0:
        new_server_only_ips = new_server_only_ips + [ipList[i]]
    else:
        new_client_only_ips = new_client_only_ips + [ipList[i]]
        
        

In [20]:
# new_server_only_ips= []
# new_client_only_ips= []


# current_region_total = 0
# for i in range(n_regions):
#     print(current_region_total,current_region_total+ clusters_per_region_map[i] *replica_count)
#     new_server_only_ips = new_server_only_ips + ipList[current_region_total:current_region_total+ clusters_per_region_map[i] *replica_count]
    
#     new_client_only_ips = new_client_only_ips + ipList[current_region_total+ clusters_per_region_map[i] *replica_count:
#                                                       current_region_total+ clusters_per_region_map[i] *(replica_count+1)]
    
    
#     current_region_total = current_region_total + clusters_per_region_map[i] *replica_count +  clusters_per_region_map[i]
    
    

In [21]:
new_server_only_ips, len(new_server_only_ips)

(['10.138.0.85',
  '10.138.0.126',
  '10.138.0.96',
  '10.138.0.109',
  '10.138.0.101',
  '10.138.0.79',
  '10.138.0.74',
  '10.138.0.69',
  '10.138.0.122',
  '10.138.0.80',
  '10.138.0.64',
  '10.138.0.87',
  '10.138.15.225',
  '10.138.0.73',
  '10.138.0.98',
  '10.138.15.201',
  '10.138.0.107',
  '10.138.15.202',
  '10.138.0.114',
  '10.138.15.213',
  '10.138.0.84',
  '10.138.0.75',
  '10.138.0.81',
  '10.138.0.102',
  '10.138.0.77',
  '10.138.0.71',
  '10.138.15.221',
  '10.138.0.105',
  '10.138.0.104',
  '10.138.0.100',
  '10.138.0.70',
  '10.138.0.95',
  '10.138.15.214',
  '10.138.15.215',
  '10.138.0.97',
  '10.138.0.111',
  '10.138.0.65',
  '10.138.15.195',
  '10.138.0.108',
  '10.138.0.78',
  '10.138.0.72',
  '10.138.0.82',
  '10.138.0.121',
  '10.138.0.124',
  '10.138.0.67',
  '10.138.0.83',
  '10.138.0.66',
  '10.138.15.216'],
 48)

In [22]:
new_client_only_ips, len(new_client_only_ips) 

(['10.138.15.194',
  '10.138.0.110',
  '10.138.0.103',
  '10.138.15.219',
  '10.138.0.99',
  '10.138.15.217',
  '10.138.0.68',
  '10.138.0.106',
  '10.138.0.86',
  '10.138.15.223',
  '10.138.15.222',
  '10.138.0.76'],
 12)

In [23]:
ipList = new_server_only_ips + new_client_only_ips

In [24]:
ipList, len(ipList)

(['10.138.0.85',
  '10.138.0.126',
  '10.138.0.96',
  '10.138.0.109',
  '10.138.0.101',
  '10.138.0.79',
  '10.138.0.74',
  '10.138.0.69',
  '10.138.0.122',
  '10.138.0.80',
  '10.138.0.64',
  '10.138.0.87',
  '10.138.15.225',
  '10.138.0.73',
  '10.138.0.98',
  '10.138.15.201',
  '10.138.0.107',
  '10.138.15.202',
  '10.138.0.114',
  '10.138.15.213',
  '10.138.0.84',
  '10.138.0.75',
  '10.138.0.81',
  '10.138.0.102',
  '10.138.0.77',
  '10.138.0.71',
  '10.138.15.221',
  '10.138.0.105',
  '10.138.0.104',
  '10.138.0.100',
  '10.138.0.70',
  '10.138.0.95',
  '10.138.15.214',
  '10.138.15.215',
  '10.138.0.97',
  '10.138.0.111',
  '10.138.0.65',
  '10.138.15.195',
  '10.138.0.108',
  '10.138.0.78',
  '10.138.0.72',
  '10.138.0.82',
  '10.138.0.121',
  '10.138.0.124',
  '10.138.0.67',
  '10.138.0.83',
  '10.138.0.66',
  '10.138.15.216',
  '10.138.15.194',
  '10.138.0.110',
  '10.138.0.103',
  '10.138.15.219',
  '10.138.0.99',
  '10.138.15.217',
  '10.138.0.68',
  '10.138.0.106',
  '10.1

In [25]:
clusterIpMap = {}

for i in range(len(ipList)):
    clusterIpMap[i] = ipList[i]

    
for i in range(nclients):
    
    clusterIpMap[7001+i] = ipList[-(nclients-i)]

In [26]:
clusterPortMap1 = {}
clusterPortMap2 = {}

for i in range(len(ipList) - nclients):
    
    clusterPortMap1[i] = str(10000)
    
for i in range(len(ipList) - nclients):
    clusterPortMap2[i] = str(20000)
    
# clusterPortMap1[7003] = '11400'
# clusterPortMap2[7003] = '11410'



temp = 11900

for i in range(nclients):
    
    clusterPortMap1[7001+i] = str(10000)
    clusterPortMap2[7001+i] = str(20000)

In [27]:
clusterPortMap1, clusterPortMap2

({0: '10000',
  1: '10000',
  2: '10000',
  3: '10000',
  4: '10000',
  5: '10000',
  6: '10000',
  7: '10000',
  8: '10000',
  9: '10000',
  10: '10000',
  11: '10000',
  12: '10000',
  13: '10000',
  14: '10000',
  15: '10000',
  16: '10000',
  17: '10000',
  18: '10000',
  19: '10000',
  20: '10000',
  21: '10000',
  22: '10000',
  23: '10000',
  24: '10000',
  25: '10000',
  26: '10000',
  27: '10000',
  28: '10000',
  29: '10000',
  30: '10000',
  31: '10000',
  32: '10000',
  33: '10000',
  34: '10000',
  35: '10000',
  36: '10000',
  37: '10000',
  38: '10000',
  39: '10000',
  40: '10000',
  41: '10000',
  42: '10000',
  43: '10000',
  44: '10000',
  45: '10000',
  46: '10000',
  47: '10000',
  7001: '10000',
  7002: '10000',
  7003: '10000',
  7004: '10000',
  7005: '10000',
  7006: '10000',
  7007: '10000',
  7008: '10000',
  7009: '10000',
  7010: '10000',
  7011: '10000',
  7012: '10000'},
 {0: '20000',
  1: '20000',
  2: '20000',
  3: '20000',
  4: '20000',
  5: '20000',
 

In [28]:
os.system('echo 3108 | sudo -S ./gradlew installDist')

[sudo] password for tejas: 


> Task :compileJava


Note: /home/tejas/work/BftSmart_Final/src/main/java/bftsmart/reconfiguration/ViewController.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.



> Task :processResources NO-SOURCE
> Task :classes
> Task :jar
> Task :installDist

BUILD SUCCESSFUL in 43s
3 actionable tasks: 3 executed


0

In [ ]:
for i in range(total_count):
    os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(i)+' 1024')

    

os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7002)+' 1024')
os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7001)+' 1024')

In [ ]:
os.system('echo 3108 | sudo -S ./gradlew clean')

In [ ]:
for key in clusterPortMap1.keys():
    if key>7002:
        print(key)
        print()
        
        for folder in os.listdir('config/'):
            if os.path.isdir('config/'+folder) and 'keys' in folder:
                if os.path.exists('config/'+folder+'/'+'privatekey7001'):
                    shutil.copy('config/'+folder+'/'+'privatekey7001','config/'+folder+'/'+'privatekey'+str(key))
                    shutil.copy('config/'+folder+'/'+'publickey7001','config/'+folder+'/'+'publickey'+str(key))
                
        print('----------')
        print()

In [ ]:
for i in range(n_clusters):
    
    if os.path.exists('config'+str(i)):
        shutil.rmtree('config'+str(i))
    
    os.mkdir('config'+str(i))
    os.mkdir('config'+str(i)+'/keysRSA')
    os.mkdir('config'+str(i)+'/keysSunEC')
    os.mkdir('config'+str(i)+'/keysECDSA')
    
    
    shutil.copytree('config/workloads/', 'config'+str(i)+'/workloads')
    shutil.copytree('config/keysSSL_TLS/', 'config'+str(i)+'/keysSSL_TLS')
    
    
    for file in os.listdir('config/keys/'):
    
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysRSA/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysSunEC/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysECDSA/'+file)
    
    
    shutil.copy('config/system.config', 'config'+str(i)+'/system.config')    
    shutil.copy('config/logback.xml', 'config'+str(i)+'/logback.xml')
    
    
    
    

In [ ]:
        
        for i in range(nclients):
            print(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

In [ ]:
for n_cluster in range(n_clusters):
    
    with open('config'+str(n_cluster)+'/hosts.config','w') as file:
        
        for n_cluster1 in range(n_clusters):
        
            for i_replica in range(replica_count):
    #             file.writelines(str(i_replica)+' '+clusterIpMap[n_cluster*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster*replica_count+i_replica]+'\n')
                file.writelines(str(n_cluster1*replica_count+i_replica)+' '+clusterIpMap[n_cluster1*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster1*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster1*replica_count+i_replica]+' '+str(n_cluster1)+'\n')

        for iter_ in range(len(ipList) - (n_clusters*replica_count)-nclients):
            print((n_clusters*replica_count) + iter_)
            
#             file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(iter_%n_clusters)+'\n')
            file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(0%n_clusters)+'\n')
        
        for i in range(nclients):
            file.writelines(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

        
        file.close()


In [ ]:
replica_count

In [ ]:
for n_cluster in range(n_clusters):
    with open('config'+str(n_cluster)+'/system.config','r') as file:
        data = file.readlines()
        f = int(np.ceil((replica_count-1)/3))
        print('f is',f )
        
        for n_line in range(len(data)):
            if ('system.servers.num' in data[n_line]) and '#' not in data[n_line]:
                
                data[n_line] = 'system.servers.num = '+str(replica_count)
                
            if ('system.servers.f' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.servers.f = '+str(f)
                
            if ('system.initial.view' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.initial.view = '
                for iter_view in range(replica_count):
                    data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
#                     data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
                
                data[n_line] = data[n_line][:-1]
                
                
        file.close()
        
    with open('config'+str(n_cluster)+'/system.config', 'w') as f:
        for line in data:
            f.write(f"{line}\n")
        

In [ ]:
for n_cluster in range(n_clusters):
    for i in range(400):
        
        for ncls in  range(1):
        
        
            print('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'privatekey'+str(1000+(ncls*100)+i))
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'publickey7002')
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'privatekey7002')

In [ ]:
# os.system('git remote set-url origin https://github.com/tmane002/BftSmart_Final.git')

In [ ]:
os.system('git add .; git commit -m "Working with modified ClusterInfo";git push')

In [ ]:
# 'gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-central1-a --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any'

In [ ]:
# gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-west1-b --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any